In [0]:
from pyspark.ml.feature import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml import *
from pyspark.ml.classification import *
from pyspark.ml.evaluation import *
from pyspark.ml.tuning import *
from pyspark.ml.regression import *
from pyspark.ml.feature import *
from pyspark.ml import *

In [0]:
ml_df = spark.table("airbnb1.gold.ml_df")
display(ml_df)

### Ordinal Encoding

In [0]:
ml_df = ml_df.withColumn("room_type_ord",
                         when(col("room_type")=="Shared room",1)\
                         .when(col("room_type")=="Private room",2)\
                         .when(col("room_type")=="Entire home/apt",3)\
                         .otherwise(0)
                         )

In [0]:
ml_df = ml_df.withColumn("property_type_ord",
                         when(col("property_type")=="Apartment",1)\
                         .when(col("property_type")=="House",2)\
                          .when(col("property_type")=="Condominium",3)\
                          .when(col("property_type")=="Townhouse",4)\
                          .when(col("property_type")=="Loft",5)\
                          .when(col("property_type")=="Bungalow",6)\
                            .otherwise(0)
                         )

In [0]:
ml_df = ml_df.drop("property_type","room_type","instant_booking_accept")
display(ml_df)

In [0]:
for i in ml_df.columns:
    x = ml_df.filter(col(i).isNull()).count()
    print(f"{i}:{x}")

#### Assembling the feaures

In [0]:
assembler = VectorAssembler(
    inputCols = ["accommodates","bedrooms","bathrooms","number_of_reviews","beds","room_type_ord","property_type_ord"],
    outputCol = "features"
)

### Feature Scaling (Standard Scaler)
Assumping it has outliers

In [0]:
scaler = StandardScaler(
    inputCol = 'features',
    outputCol = "scaled_features",
    withStd = True,
    withMean = True
)

### Splitting the dataset

In [0]:
train_df , test_df = ml_df.randomSplit([0.7,0.3],seed=42)

### Linear Regression

In [0]:
lr = LinearRegression(
  featuresCol= 'scaled_features',
  labelCol= 'price'
)

### Building a Pipeline
Instead doing fit and tranform for every step building a pipeline

In [0]:
pipeline = Pipeline(stages=[assembler,scaler,lr])
model = pipeline.fit(train_df)

#### prediction

In [0]:
predictions = model.transform(test_df)


In [0]:
predictions.select("price","room_type_ord","prediction").show(5)

### model evaluation using rmse

In [0]:
evaluation = RegressionEvaluator(
  labelCol= 'price',
  predictionCol= 'prediction',
  metricName= 'rmse'
)
rmse = evaluation.evaluate(predictions)
print(rmse)

In [0]:
e1 = RegressionEvaluator(
  labelCol= 'price',
  predictionCol= 'prediction',
  metricName= 'r2'
)
r2 = e1.evaluate(predictions)
print(r2)

r^2 value is less than 0.2 so it is very weak and rmse is also apprx 389 that means the predictions has difference of about 389$
so better to use another model like any tree based beacuse the price is non linear

#### Random Forest